In [1]:
!pip install tensorboard
!pip install tensorboardX

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


# Tensorboard

Tensorboard 是 TensorFlow 的一个附加工具，用于记录训练过程的模型的参数、评价指标与图像等细节内容，并通过 Web 页面提供查看细节与过程的功能，用浏览器可视化的形式展现，帮助我们在实验时观察神经网络的训练过程，把握训练趋势。

In [2]:
# 为了使用 TensorboardX，我们首先需要创建一个 SummaryWriter 的实例，然后再使用add_scalar方法或add_image方法，将数字或图片记录到 SummaryWriter 实例中。

import torch
from torch.utils.tensorboard import SummaryWriter

# 1) 创建 writer（建议指定目录）
writer = SummaryWriter(log_dir="runs/demo")

# 2) 记录一个标量
tag = "train/loss"
scalar_value = 0.123
writer.add_scalar(tag, scalar_value, global_step=0)

# 3) 记录一张图片（CHW）
img_tensor = torch.rand(3, 224, 224)  # 3x224x224
writer.add_image("sample/image", img_tensor, global_step=0, dataformats="CHW")

writer.close()
print("logs 写在 runs/demo")


logs 写在 runs/demo


另外一个例子:

In [3]:
from torch.utils.tensorboard import SummaryWriter
# PyTorch 1.8之前的版本请使用：
# from tensorboardX import SummaryWriter
import numpy as np

# 创建一个SummaryWriter的实例
writer = SummaryWriter()

for n_iter in range(100):  #连续100步写入4条标量曲线
    writer.add_scalar('Loss/train', np.random.random(), n_iter)     #模拟真实数据
    writer.add_scalar('Loss/test', np.random.random(), n_iter)      #模拟真实数据
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter) #模拟真实数据
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)  #模拟真实数据

#写入一张 3 通道图片
img = np.zeros((3, 100, 100))     # 形状是 C x H x W (CHW)
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000      # R通道：从黑到亮
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000     #G通道：从亮到黑

writer.add_image('my_image', img, 0)
writer.close()

In [4]:
# !tensorboard --logdir runs --port 6006
# 注意：因为这个命令会阻断后续的步骤，如果需要看图，则取消注释

进入 http://localhost:6006 后可以看到图片：

![](Images/12.png)


点击Imag 标签可以看到可以显示刚刚用add_image方法记录的图像数据：


![](Images/13.png)


### 训练过程可视化

In [5]:
import random
import numpy as np
import torch
from torch import nn

# 模型定义
class LinearModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(1))
    self.bias = nn.Parameter(torch.randn(1))

  def forward(self, input):
    return (input * self.weight) + self.bias

# 数据
w = 2
b = 3
xlim = [-10, 10]
x_train = np.random.randint(low=xlim[0], high=xlim[1], size=30)
y_train = [w * x + b + random.randint(0,2) for x in x_train]

In [6]:
# Tensorboard
from torch.utils.tensorboard import SummaryWriter

# 训练
model = LinearModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-2, momentum=0.9)
y_train = torch.tensor(y_train, dtype=torch.float32)

writer = SummaryWriter()

for n_iter in range(500):
    input = torch.from_numpy(x_train)
    output = model(input)
    loss = nn.MSELoss()(output, y_train)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    writer.add_scalar('Loss/train', loss, n_iter)

In [7]:
# !tensorboard --logdir runs --port 6006
# 注意：因为这个命令会阻断后续的步骤，如果需要看图，则取消注释

进入 http://localhost:6006 后可以看到图片：

![](Images/14.png)
